<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [3]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

In [4]:
data= pd.read_csv('car_insurance.csv')

In [5]:
data.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0-9y,high school,upper class,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,0,1,0-9y,none,poverty,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,0,0,0-9y,high school,working class,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,0,1,0-9y,university,working class,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,1,1,10-19y,none,working class,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [7]:
data.isna().sum()

id                       0
age                      0
gender                   0
driving_experience       0
education                0
income                   0
credit_score           982
vehicle_ownership        0
vehicle_year             0
married                  0
children                 0
postal_code              0
annual_mileage         957
vehicle_type             0
speeding_violations      0
duis                     0
past_accidents           0
outcome                  0
dtype: int64

In [8]:
data= data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8149 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   8149 non-null   int64  
 1   age                  8149 non-null   int64  
 2   gender               8149 non-null   int64  
 3   driving_experience   8149 non-null   object 
 4   education            8149 non-null   object 
 5   income               8149 non-null   object 
 6   credit_score         8149 non-null   float64
 7   vehicle_ownership    8149 non-null   float64
 8   vehicle_year         8149 non-null   object 
 9   married              8149 non-null   float64
 10  children             8149 non-null   float64
 11  postal_code          8149 non-null   int64  
 12  annual_mileage       8149 non-null   float64
 13  vehicle_type         8149 non-null   object 
 14  speeding_violations  8149 non-null   int64  
 15  duis                 8149 non-null   i

In [9]:
data.describe(include= 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,8149.0,NaN,NaN,NaN,501339.680083,290857.727101,101.0,247706.0,503269.0,756207.0,999976.0
age,8149.0,NaN,NaN,NaN,1.499816,1.025045,0.0,1.0,1.0,2.0,3.0
gender,8149.0,NaN,NaN,NaN,0.498834,0.500029,0.0,0.0,0.0,1.0,1.0
driving_experience,8149,4,0-9y,2864,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education,8149,3,high school,3404,NaN,NaN,NaN,NaN,NaN,NaN,NaN
income,8149,4,upper class,3588,NaN,NaN,NaN,NaN,NaN,NaN,NaN
credit_score,8149.0,NaN,NaN,NaN,0.516368,0.137684,0.053358,0.417891,0.526759,0.620074,0.960819
vehicle_ownership,8149.0,NaN,NaN,NaN,0.699227,0.458622,0.0,0.0,1.0,1.0,1.0
vehicle_year,8149,2,before 2015,5642,NaN,NaN,NaN,NaN,NaN,NaN,NaN
married,8149.0,NaN,NaN,NaN,0.501043,0.50003,0.0,0.0,1.0,1.0,1.0


In [10]:
data.corr()

,id,age,gender,credit_score,vehicle_ownership,married,children,postal_code,annual_mileage,speeding_violations,duis,past_accidents,outcome
id,1.000000,0.008845,-0.000659,-0.002144,0.004358,0.009331,-0.000431,-0.001299,0.002282,0.012750,0.006082,0.006239,-0.006222
age,0.008845,1.000000,0.009158,0.495619,0.266822,0.380782,0.387673,0.008648,-0.275722,0.458088,0.283183,0.429655,-0.444137
gender,-0.000659,0.009158,1.000000,-0.078427,0.018540,0.015835,0.005859,-0.002766,-0.021560,0.207218,0.104925,0.228203,0.098576
credit_score,-0.002144,0.495619,-0.078427,1.000000,0.312995,0.282555,0.221354,0.007465,-0.174453,0.208433,0.127888,0.182408,-0.321436
vehicle_ownership,0.004358,0.266822,0.018540,0.312995,1.000000,0.170752,0.116477,-0.010469,-0.093710,0.133302,0.087432,0.121881,-0.386217
married,0.009331,0.380782,0.015835,0.282555,0.170752,1.000000,0.293613,0.010056,-0.465642,0.216213,0.119750,0.214767,-0.253200
children,-0.000431,0.387673,0.005859,0.221354,0.116477,0.293613,1.000000,0.019260,-0.451105,0.224753,0.113996,0.204888,-0.230255
postal_code,-0.001299,0.008648,-0.002766,0.007465,-0.010469,0.010056,0.019260,1.000000,-0.130169,0.113228,0.034591,-0.120617,0.095939
annual_mileage,0.002282,-0.275722,-0.021560,-0.174453,-0.093710,-0.465642,-0.451105,-0.130169,1.000000,-0.318478,-0.113474,-0.195160,0.184865
speeding_violations,0.012750,0.458088,0.207218,0.208433,0.133302,0.216213,0.224753,0.113228,-0.318478,1.000000,0.365700,0.445531,-0.293138


In [11]:
models= []

In [12]:
# Feature columns
features = data.drop(columns=["id", "outcome"]).columns

In [13]:
# Loop through features
for col in features:
    # Create a model
    model = logit(f"outcome ~ {col}", data= data).fit()
    # Add each model to the models list
    models.append(model)

Optimization terminated successfully.
         Current function value: 0.512632
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615189
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.466479
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603304
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.533082
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.566906
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548159
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572577
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.587413
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.594367
  

In [14]:
# Empty list to store accuracies
accuracies = []

# Loop through models
for feature in range(0, len(models)):
    # Compute the confusion matrix
    conf_matrix = models[feature].pred_table()
    # True negatives
    tn = conf_matrix[0,0]
    # True positives
    tp = conf_matrix[1,1]
    # False negatives
    fn = conf_matrix[1,0]
    # False positives
    fp = conf_matrix[0,1]
    # Compute accuracy
    acc = (tn + tp) / (tn + fn + fp + tp)
    accuracies.append(acc)

accuracies

[0.7740827095349122,
 0.6887961713093631,
 0.7781322861700822,
 0.6887961713093631,
 0.7399680942446926,
 0.706589765615413,
 0.7392318075837526,
 0.6887961713093631,
 0.6887961713093631,
 0.6887961713093631,
 0.6887961713093631,
 0.692723033501043,
 0.6887961713093631,
 0.6887961713093631,
 0.6887961713093631,
 0.6887961713093631]

In [16]:
# Find the feature with the largest accuracy
best_feature = features[accuracies.index(max(accuracies))]
best_feature

'driving_experience'

In [17]:
# Create best_feature_df
best_feature_df = pd.DataFrame({"best_feature": best_feature,
                                "best_accuracy": max(accuracies)},
                                index=[0])
best_feature_df

,best_feature,best_accuracy
0,driving_experience,0.778132
